# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="Banana.app")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# Agent and Network

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from collections import deque
import random
import torch.optim as optim

class Network(nn.Module):

    def __init__(self, state_size, action_size, layers, use_dueling_dqn=False):
        super().__init__()

        self.layers = nn.ModuleList([nn.Linear(state_size, layers[0])])
        self.layers.extend([nn.Linear(i, o) for i, o in zip(layers[:-1], layers[1:])])
        self.use_dueling_dqn = use_dueling_dqn
        self.state_size = state_size
        self.action_size = action_size

        if use_dueling_dqn:
            self.advantage = nn.Sequential(
                nn.Linear(layers[-1], layers[-1]),
                nn.ELU(),
                nn.Linear(layers[-1], action_size)
            )
            self.value = nn.Sequential(
                nn.Linear(layers[-1], layers[-1]),
                nn.ELU(),
                nn.Linear(layers[-1], 1)
            )

        self.output = nn.Linear(layers[-1], action_size)
    
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.elu(layer(x))

        if self.use_dueling_dqn:
            advantage = self.advantage(x)
            value = self.value(x)
            return value + advantage - advantage.mean()

        return self.output(x)
    
    def save(self, filepath='checkpoint.pth'):
        checkpoint = {
            'state_size': self.state_size,
            'action_size': self.action_size,
            'layers': [each.out_features for each in self.layers],
            'use_dueling_dqn': self.use_dueling_dqn,
            'state_dict': self.state_dict()
        }
        torch.save(checkpoint, filepath)

    # static method to load saved network
    def load(filepath='checkpoint.pth'):
        state_size, action_size, layers, use_dueling_dqn=False
        checkpoint = torch.load(filepath)
        network = Network(
            checkpoint['state_size'],
            checkpoint['action_size'],
            layers=checkpoint['layers'],
            use_dueling_dqn=checkpoint['use_dueling_dqn']
        )
        network.load_state_dict(checkpoint['state_dict'])
        return network

class ReplayBuffer():

    def __init__(self, buffer_size):
        self.memory = deque(maxlen=buffer_size)
    
    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        experiences = random.sample(self.memory, k=batch_size)
        experiences = [np.array(a) for a in zip(*experiences)]
        return experiences
    
    def __len__(self):
        return len(self.memory)
    
class Agent():

    def __init__(
        self, 
        state_size,
        action_size, 
        use_double_dqn=False,
        use_dueling_dqn=False,
        hidden_layers=[64, 64], # neural network hidden layers
        update_every=4, # how often to update the network
        batch_size=64, # minibatch size
        tau=1e-3, # for soft update of target parameters
        lr=5e-4, # learning rate 
        gamma=0.99, # discount factor
        buffer_size=int(1e5), # replay buffer size
        epsilon=1.0, # exploration probability
        epsilon_decay=0.97,
        epsilon_min=0.02
    ):

        self.buffer_size = buffer_size 
        self.gamma = gamma 
        self.tau = tau
        self.lr = lr
        self.update_every = update_every 
        self.batch_size = batch_size 
        
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        self.use_double_dqn = use_double_dqn
        self.use_dueling_dqn = use_dueling_dqn

        self.hidden_layers = hidden_layers
        self.action_size = action_size
        self.state_size = state_size
        self.qnetwork_local = Network(state_size, action_size, hidden_layers, use_dueling_dqn)
        self.qnetwork_target = Network(state_size, action_size, hidden_layers, use_dueling_dqn)
        self.memory = ReplayBuffer(self.buffer_size)
        self.t_step = 0
        
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)
        
    def load(self, filepath):
        self.qnetwork_local = Network.load(filepath)
        self.qnetwork_target = Network.load(filepath)
        
    def save(self, filepath):
        self.qnetwork_local.save(filepath)

    def act(self, state):
        probs = self.get_probs(state)
        action = np.random.choice(self.action_size, p=probs)
        return action
    
    def get_probs(self, state):
        probs = np.full(action_size, self.epsilon / self.action_size)
        best_action = self.get_best_action(state)
        probs[best_action] += 1 - self.epsilon
        return probs
    
    def get_best_action(self, state):
        action = None

        self.qnetwork_local.eval()
        with torch.no_grad():
            state = torch.tensor(state).float().unsqueeze(0)
            best = self.qnetwork_local.forward(state).max(1)
            action = best[1].item()
        self.qnetwork_local.train()

        return action
    
    def step(self, state, action, reward, next_state, done):
        pass
        self.t_step += 1
        self.memory.add(state, action, reward, next_state, done)
        
        self.t_step %= self.update_every
        if self.t_step == 0 and len(self.memory) >= self.batch_size:
            self.learn()
            
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon_decay*self.epsilon)
    
    def learn(self):
        experiences = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = experiences
        
        states = torch.FloatTensor(states)
        next_states = torch.FloatTensor(next_states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones.astype(np.uint8))
        
        q_values = self.qnetwork_local(states)
        q_values_next = self.qnetwork_local(next_states)
        q_values_next_ = self.qnetwork_target(next_states)
        
        Q_targets_next = None
        if self.use_double_dqn:
            best_next_actions = q_values_next.detach().max(1)[1].unsqueeze(1)
            Q_targets_next = q_values_next_.gather(1, best_next_actions).squeeze()
        else:
            Q_targets_next = q_values_next_.detach().max(1)[0]
        
        Q_targets = rewards + self.gamma * Q_targets_next * (1 - dones)
        Q_expected = q_values.gather(1, actions.unsqueeze(1)).squeeze()

        self.optimizer.zero_grad()
        loss = F.mse_loss(Q_expected, Q_targets)
        loss.backward()
        self.optimizer.step()
        
        self.soft_update()
        
    def soft_update(self):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target, local in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
            target.data.copy_(self.tau * local.data + (1 - self.tau) * target.data)

# Training the Agent

Next, start the environment with graphics turned off so that the agent can train without the overehead of rendering the display.

In [4]:
env = UnityEnvironment(file_name="Banana.app", no_graphics=True)
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

def train(agent, label='agent', episodes=1000, plot_every=10, print_every=100, stop_when_solved=True):
    
    scores = []
    tmp_scores = deque(maxlen=100)

    for episode in range(1, episodes+1):

        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        average = 0
        best = 0

        while True:

            action = agent.act(state)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]

            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward

            if done:
                tmp_scores.append(score)
                average = np.mean(tmp_scores)
                if average > best:
                    best = average
                    agent.save("checkpoints/{}.pth".format(label))

                if episode % plot_every == 0:
                    scores.append((episode, average))
                if episode % print_every == 0:
                    print("Episode: {}\tAverage Score: {:.2f}".format(episode, average))
                break

        # solved if average over last 100 episodes is at least 13
        if average >= 13.0:
            print("Environment solved in {} episodes!\tAverage Score: {:.2f}".format(episode, average))
            if stop_when_solved:
                break

    return scores

def plot_scores(scores):
    for label in scores:
        x, y = zip(*scores[label])
        plt.plot(x, y, label=label)
    plt.legend()
    plt.xlabel('Episode')
    plt.ylabel('Average Score')
    plt.show()

# Exploring Architecture and Parameters

Next I explore different network architectures and hyperparameters to find an agent that learns the environment efficiently.  Because of the time required to test each agent, I do not try all the combinations of each parameter, but instead evaluate each parameter while leaving the rest set to the the defaults.  I also quit as soon as the environment is solved, so it is possible that some of these parameters work well early on but do not do as well if allowed to train longer.

In [ ]:
agents = {
    "DQN": Agent(state_size, action_size),
    "Dueling DQN": Agent(state_size, action_size, use_dueling_dqn=False),
    "DDQN": Agent(state_size, action_size, use_double_dqn=True),
    "Dueling DDQN": Agent(state_size, action_size, use_dueling_dqn=True, use_double_dqn=True),
    "Layers=128,64": Agent(state_size, action_size, hidden_layers=[128,64]),
    "Layers=64,32": Agent(state_size, action_size, hidden_layers=[64,32]),
    "Layers=32,32": Agent(state_size, action_size, hidden_layers=[32,32]),
    "Gamma=1.0": Agent(state_size, action_size, gamma=1.0),
    "Gamma=.95": Agent(state_size, action_size, gamma=0.95),
    "LR=1e-3": Agent(state_size, action_size, lr=1e-3),
    "LR=1e-4": Agent(state_size, action_size, lr=1e-4),
    "Tau=1e-1": Agent(state_size, action_size, tau=1e-1),
    "Tau=1e-2": Agent(state_size, action_size, tau=1e-2),
    "Tau=1e-4": Agent(state_size, action_size, tau=1e-4),
    "EpsilonMin=.01": Agent(state_size, action_size, epsilon_min=0.01),
    "EpsilonDecay=.98": Agent(state_size, action_size, epsilon_decay=0.98),
    "EpsilonDecay=.99": Agent(state_size, action_size, epsilon_decay=0.99),
    "Buffer=1e4": Agent(state_size, action_size, buffer_size=int(1e4)),
    "Buffer=1e6": Agent(state_size, action_size, buffer_size=int(1e6)),
}
scores = {}
for agent in agents:
    print("Training Agent: {}".format(agent))
    scores[agent] = train(agents[agent], label=agent)
plot_scores(scores)

Training Agent: DQN
Episode: 100	Average Score: 3.54
Episode: 200	Average Score: 8.75
Episode: 300	Average Score: 11.95
Environment solved in 336 episodes!	Average Score: 13.01
Training Agent: Dueling DQN
Episode: 100	Average Score: 1.39
Episode: 200	Average Score: 7.74
Episode: 300	Average Score: 11.77
Environment solved in 316 episodes!	Average Score: 13.10
Training Agent: DDQN
Episode: 100	Average Score: 3.96
Episode: 200	Average Score: 10.27
Episode: 300	Average Score: 11.88
Environment solved in 379 episodes!	Average Score: 13.12
Training Agent: Dueling DDQN
Episode: 100	Average Score: 4.94
Episode: 200	Average Score: 9.93
Environment solved in 273 episodes!	Average Score: 13.04
Training Agent: Layers=128,64
Episode: 100	Average Score: 1.37
Episode: 200	Average Score: 4.25
Episode: 300	Average Score: 9.25
Episode: 400	Average Score: 12.83
Environment solved in 406 episodes!	Average Score: 13.03
Training Agent: Layers=64,32
Episode: 100	Average Score: 2.24
Episode: 200	Average Scor

# Trained Agent

After finding an architecture and hyperparameters that learn the environment efficiently, I train over more episodes to get an agent that performs even better than the initial solution.  Checkpoints save each time the running average over 100 episodes is increased.

In [ ]:
print('Training Agent')
agent = Agent(state_size, action_size)
score = train(agent, label='final', episodes=2000, stop_when_solved=False)
print('Done')

In [ ]:
env = UnityEnvironment(file_name="Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
state_size = brain.vector_observation_space_size
action_size = brain.vector_action_space_size

In [7]:
agent = Agent(state_size, action_size, epsilon=0)
agent.load("checkpoints/architectures_dqn.pth")

env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
score = 0

while True:

    action = agent.act(state)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]

    state = next_state
    score += reward

    if done:
        break
        
print("Score: {}".format(score))

Score: 1.0
